import time
plc=1
time.sleep(3000*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization,Input,ReLU,add,MaxPool1D,GlobalAveragePooling1D
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score,mean_absolute_error
from tensorflow.keras import callbacks,Model,layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
cwd = os.getcwd() #
fullscrn()

In [ ]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

def wndwng0d(x,snstvty=2):            #روش تشخیص ایپاک پایه
    smpl_rte=480
    i=0
    wndws=np.array([])
    #for i in range (1,2):   #print(np.shape(x))
    if snstvty==1 :                        #تشخیص ایپاک پایه
        xj=np.array(x)
        peaks,_ = find_peaks(xj, height=0.03*max(xj), distance=200,prominence=300, width=3)
        if len(peaks)<6 :
            peaks,_ = find_peaks(xj, height=0.03*max(xj), distance=160,prominence=100, width=3)
            #print('class ', i, 'Row = ', j, 'have ',len(peaks),'peaks')
        if len(peaks)>1 :
            for z in range (len(peaks)):
                if (peaks[z]<3100 and peaks[z]!=peaks[-1]) :
                    win=np.array(xj[peaks[z]:peaks[z+1]])
                    win = signal.resample(win, smpl_rte)
                    if len(win)==smpl_rte :  #win=np.append(win, i)# if len(win)==smpl_rte+1 :
                        wndws=np.append(wndws, win)


    if snstvty==2 :                        #تشخیص ایپاک حساس تر
        xj=np.array(x)
        peaks,_ = find_peaks(xj, height=0.02*max(xj), distance=160,prominence=200, width=3)
        if len(peaks)<6 :
            peaks,_ = find_peaks(xj, height=0.01*max(xj), distance=120,prominence=60, width=3)
        if len(peaks)>1 :
            for z in range (len(peaks)):
                if (peaks[z]<3100 and peaks[z]!=peaks[-1]) :
                    win=np.array(xj[peaks[z]:peaks[z+1]])
                    win = signal.resample(win, smpl_rte)
                    if len(win)==smpl_rte :
                        wndws=np.append(wndws, win)
    rows=int(len(wndws)/(smpl_rte))            #/(smpl_rte+1))
    wndws0=np.reshape(wndws,(rows,(smpl_rte)))    #(wndws,(rows,(smpl_rte+1)))
    return(wndws0)

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
cls_num=17
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)]=MITBIH(i)
os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,cls_num+1):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print( i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)
btch=1000

first_train_shape= (276, 81)
first_test_shape= (600, 81)
classes_quantity= 2
tr_lbls=	 {0, 1}
Count_labels= [161]
max(train_feature_Altitude)= 2.4602
min(train_feature_Altitude)= -2.1799
first_train_sample=
 [ 1.       -0.88425  -0.76511  -0.6264   -0.30459  -0.014302  0.29922
  0.51262   0.62572   0.89455   1.053     1.1322    1.2672    1.4321
  1.4845    1.2757    1.1589    1.0053    0.81736   0.50439   0.37885
  0.14581  -0.20947  -0.57872  -0.67481  -0.79999  -0.93504  -0.78816
 -0.57744  -0.27191   0.061242  0.41037   0.76072   1.1348    1.5046
  1.6696    1.6999    1.5531    1.3937    1.4704    1.5877    1.6902
  1.7647    1.5246    1.1461    0.76139   0.42848   0.076242 -0.2335
 -0.51671  -0.81834  -0.080823 -0.27898  -0.46208  -0.69433  -0.92273
 -1.1133   -1.3737   -1.5067   -1.5621   -1.3708   -1.102    -0.86298
 -0.53403  -0.3252   -0.1899   -0.3366   -0.45422  -0.60653  -0.78383
 -1.0497   -1.3447   -1.4977   -1.5734   -1.4749   -1.2418   -1.0149
 -0.78176  -0.61354  -0.306

In [3]:
np.shape(ecg)

(276, 481)

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [5]:
smpl_rte=480                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])
for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    dta=np.array(vars()['ecg'+str(cls)][int(.1*len(vars()['ecg'+str(cls)])):,:]) #انتخاب محدوده ترِین جهت داده افزایی
    vars()['wndws'+str(cls)]=wndwng(dta)                #نرخ نمونه برداری 480 است و شماره کلاس در آخر ردیف نیست
    clm0vlu=np.zeros(len(vars()['wndws'+str(cls)]))
    cls_clm=np.array(clm0vlu)
    cls_clm=np.int16(cls_clm+cls)
    cls_clm=np.transpose([cls_clm])                     # افزودن ستون برچسب (شماره کلاس) به سمت راست پنجره ها
    vars()['wndws'+str(cls)]=np.concatenate((vars()['wndws'+str(cls)],cls_clm),axis=1)   

print("each class and its windows =")
cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
cls_wndws=np.int16(cls_wndws)    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
0 115	1 161	
 max = 161


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [6]:
smpl_rte=len(wndws1[0])                                        # در ماژول ها نیز همین مقدار ثبت شده

#add augmented data to base data
xtrain=np.empty((0,len(wndws1[0])), float) 
for i in range (0,cls_num):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))

Up to class  0 train shape =  (115, 481)
Up to class  1 train shape =  (276, 481)


In [7]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[115 161]


In [8]:
xtrain[:,-1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [9]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (0,cls_num):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [10]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [11]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (0,cls_num):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [12]:
#print(np.max(wndws_tst))
for i in range (0,cls_num):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (0,cls_num):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 0  >> 
max magnitude class 0  =  2.4659367123545555
min magnitude class 0  =  -2.1616024973492727
after normalizing >>
max magnitude class 0  =  0.9764289207494858
min magnitude class 0  =  -0.9718534363802719

 cls 1  >> 
max magnitude class 1  =  2.095753113350402
min magnitude class 1  =  -1.994499999999999
after normalizing >>
max magnitude class 1  =  0.8205745579542003
min magnitude class 1  =  -0.9015000946440819
each class and its windows =
0 222	1 378	
 max instance in classes of test windows = 378


In [13]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -0.9718534363802719
max tst = 0.9764289207494858


In [14]:
len(wndws_test)

600

In [15]:
np.shape(wndws_test)

(600, 481)

In [16]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [17]:
fullprint(ytest)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [18]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [19]:
np.shape(wndws1)

(161, 481)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [20]:
Xvalid=np.array(Xtrain)
yvalid=np.array(ytrain)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [21]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtrain)
y_valid=np.array(ytrain)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (276, 480)
y_train =>  (276,)
X_test  =>  (600, 480)
y_test  =>  (600,)
X_valid  =>  (276, 480)
y_valid  =>  (276,)


In [22]:
if min(y_train)==0:
    print(' class was started from 0=Zero ')
else:
    print('!!!!!! class Should start from 0=Zero !!!!!!')
    print("!!!!!!   Reduce 1 From Class Number    !!!!!")
    y_train=np.array(y_train-1)
    y_test=np.array(y_test-1)
    y_valid=np.array(y_valid-1)
print('final labels=',set(y_train))

 class was started from 0=Zero 
final labels= {0, 1}


In [23]:
y_train_cat=to_categorical(y_train)#[:,1:]
y_valid_cat=to_categorical(y_valid)#[:,1:]
y_test_cat=to_categorical(y_test)#[:,1:]


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

<h1><center><div style="direction:rtl;font-family:B Nazanin">Network</div></center></h1>

In [24]:
# Import ResNet1D Module
#from ResNet_1DCNN import ResNet
#from ResNeXt_1DCNN import ResNeXt as ResNet
from SE_ResNeXt_1DCNN import SEResNeXt as ResNet

"Configurations for ResNet in Regression Mode"
length = X_train.shape[1]   # Number of Features (or length of the signal)
model_width = 32           # Number of Filter or Kernel in the Input Layer
num_channel = 1             # Number of Input Channels
problem_type = 'Classification' # Regression or Classification
output_number = cls_num     # Number of Outputs in the Regression Mode
Regression_Model = ResNet(length, num_channel, model_width, problem_type=problem_type, output_nums=output_number).SEResNeXt18() # Build Model
# SEResNext Models supported: SEResNeXt18, SEResNeXt34, SEResNeXt50, SEResNeXt101, SEResNeXt152, 
Regression_Model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 100, restore_best_weights = True)
history = Regression_Model.fit(X_train, y_train_cat, epochs=500, batch_size=btch, verbose=1, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping]) #,validation_split=0.2,callbacks =[callbacks]

Epoch 1/500
14/14 [==============================] - 30s 685ms/step - loss: 0.8196 - accuracy: 0.6703 - val_loss: 1.2381 - val_accuracy: 0.4167
Epoch 2/500
14/14 [==============================] - 7s 491ms/step - loss: 0.6430 - accuracy: 0.6848 - val_loss: 1.3200 - val_accuracy: 0.5833
Epoch 3/500
14/14 [==============================] - 8s 636ms/step - loss: 0.5827 - accuracy: 0.7355 - val_loss: 0.6792 - val_accuracy: 0.5036
Epoch 4/500
14/14 [==============================] - 6s 483ms/step - loss: 0.5709 - accuracy: 0.7138 - val_loss: 0.7623 - val_accuracy: 0.4058
Epoch 5/500
14/14 [==============================] - 6s 478ms/step - loss: 0.4921 - accuracy: 0.7935 - val_loss: 1.4413 - val_accuracy: 0.4167
Epoch 6/500
14/14 [==============================] - 6s 482ms/step - loss: 0.4641 - accuracy: 0.8116 - val_loss: 1.2906 - val_accuracy: 0.4167
Epoch 7/500
14/14 [==============================] - 9s 652ms/step - loss: 0.3843 - accuracy: 0.8514 - val_loss: 1.6094 - val_accuracy: 0.416

Epoch 58/500
14/14 [==============================] - 8s 484ms/step - loss: 0.0876 - accuracy: 0.9638 - val_loss: 0.0533 - val_accuracy: 0.9783
Epoch 59/500
14/14 [==============================] - 6s 485ms/step - loss: 0.0514 - accuracy: 0.9746 - val_loss: 0.0600 - val_accuracy: 0.9819
Epoch 60/500
14/14 [==============================] - 6s 479ms/step - loss: 0.0352 - accuracy: 0.9891 - val_loss: 0.8002 - val_accuracy: 0.8043
Epoch 61/500
14/14 [==============================] - 9s 648ms/step - loss: 0.0846 - accuracy: 0.9746 - val_loss: 0.0844 - val_accuracy: 0.9855
Epoch 62/500
14/14 [==============================] - 6s 479ms/step - loss: 0.0556 - accuracy: 0.9928 - val_loss: 0.0410 - val_accuracy: 0.9855
Epoch 63/500
14/14 [==============================] - 6s 477ms/step - loss: 0.0606 - accuracy: 0.9746 - val_loss: 1.5484 - val_accuracy: 0.7319
Epoch 64/500
14/14 [==============================] - 6s 479ms/step - loss: 0.0404 - accuracy: 0.9855 - val_loss: 0.2998 - val_accuracy:

Epoch 115/500
14/14 [==============================] - 3s 181ms/step - loss: 0.0565 - accuracy: 0.9891 - val_loss: 0.2982 - val_accuracy: 0.9348
Epoch 116/500
14/14 [==============================] - 5s 336ms/step - loss: 0.0391 - accuracy: 0.9855 - val_loss: 0.0026 - val_accuracy: 1.0000
Epoch 117/500
14/14 [==============================] - 9s 646ms/step - loss: 0.0083 - accuracy: 0.9964 - val_loss: 0.1222 - val_accuracy: 0.9601
Epoch 118/500
14/14 [==============================] - 6s 476ms/step - loss: 0.0203 - accuracy: 0.9928 - val_loss: 0.0115 - val_accuracy: 0.9964
Epoch 119/500
14/14 [==============================] - 6s 481ms/step - loss: 0.0275 - accuracy: 0.9855 - val_loss: 0.0704 - val_accuracy: 0.9746
Epoch 120/500
14/14 [==============================] - 6s 477ms/step - loss: 0.0068 - accuracy: 1.0000 - val_loss: 5.1113e-04 - val_accuracy: 1.0000
Epoch 121/500
14/14 [==============================] - 8s 482ms/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 6.2722e-05 

In [25]:
predict_x=Regression_Model.predict(X_test)              # Function 1

y_pred_crs=np.argmax(predict_x,axis=1)                 # function from augment.py to remove 0 index predictions

#y_pred_crs = model_Res.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

19/19 [==============================] - 4s 34ms/step
array([[150,  72],
       [ 73, 305]], dtype=int64)
              precision    recall  f1-score   support

           0       0.67      0.68      0.67       222
           1       0.81      0.81      0.81       378

    accuracy                           0.76       600
   macro avg       0.74      0.74      0.74       600
weighted avg       0.76      0.76      0.76       600

